In [12]:
import numpy as np
from scipy import integrate
from scipy.linalg import expm 

In [13]:
#ПАРАМЕТРЫ МОЕДЕЛИ#
J = -1.0
g = 0.5
chi = 2

d = 2
delta = 0.005
N = 200

In [14]:
# Инициализация начального состояние MPS - Придерживаемся канонической формы! САЙТЫ A B - записываются в один массив
B = []
s = []
for i in range(2):
    B.append(np.zeros([2, 1, 1]))
    B[-1][0, 0, 0] = 1
    s.append(np.ones([1]))

In [15]:
#ЗАДАНИЕ ОПЕРАТОРА ЭВОЛЮЦИИ#
H_bond = np.array([[J, -g / 2, -g / 2, 0], 
                   [-g / 2, -J, 0, -g / 2],
                   [-g / 2, 0, -J, -g / 2], 
                   [0, -g / 2, -g / 2, J]])
U = np.reshape(expm(-delta * H_bond), (2, 2, 2, 2))

In [43]:
for step in range(0, N):
    for i_bond in [0, 1]:

        ia = np.mod(i_bond - 1, 2)
        ib = np.mod(i_bond, 2)
        ic = np.mod(i_bond + 1, 2)
        chia = B[ib].shape[1]
        chic = B[ic].shape[2]

        #СВЁРТКА ЛОКАЛЬНЫХ ТЕНЗОРОВ A  - B #
        theta = np.tensordot(B[ib], B[ic], axes=(2, 1))  # i a j b
        theta = np.tensordot(U, theta, axes=([2, 3], [0, 2]))  # ip jp a b
        theta = np.tensordot(np.diag(s[ia]), theta, axes=([1, 2]))  # a ip jp b
        theta = np.reshape(np.transpose(theta, (1, 0, 2, 3)),
                           (d * chia, d * chic))  # ip a jp b

        # SVD - Разложение Шмидта #
        X, Y, Z = np.linalg.svd(theta, full_matrices=0)
        chi2 = np.min([np.sum(Y > 10.**(-10)), chi])

        piv = np.zeros(len(Y), np.bool)
        piv[(np.argsort(Y)[::-1])[:chi2]] = True

        Y = Y[piv]
        invsq = np.sqrt(sum(Y**2))
        X = X[:, piv]
        Z = Z[piv, :]

        # Обновление состояния#
        s[ib] = Y / invsq

        X = np.reshape(X, (d, chia, chi2))
        X = np.transpose(np.tensordot(np.diag(s[ia]**(-1)), X, axes=(1, 1)),
                         (1, 0, 2))
        B[ib] = np.tensordot(X, np.diag(s[ib]), axes=(2, 0))

        B[ic] = np.transpose(np.reshape(Z, (chi2, d, chic)), (1, 0, 2))

In [44]:
E = []
for i_bond in range(2):
    BB = np.tensordot(B[i_bond], B[np.mod(i_bond + 1, 2)], axes=(2, 1))
    sBB = np.tensordot(np.diag(s[np.mod(i_bond - 1, 2)]), BB, axes=(1, 1))
    C = np.tensordot(sBB,
                     np.reshape(H_bond, [d, d, d, d]),
                     axes=([1, 2], [2, 3]))
    sBB = np.conj(sBB)
    E.append(
        np.squeeze(np.tensordot(sBB, C, axes=([0, 3, 1, 2], [0, 1, 2,
                                                             3]))).item())
print("E_iTEBD =", np.mean(E))

f = lambda k, g: -2 * np.sqrt(1 + g**2 - 2 * g * np.cos(k)) / np.pi / 2.
E0_exact = integrate.quad(f, 0, np.pi, args=(g, ))[0]
print("E_exact =", E0_exact)
print('de = ', (np.mean(E) - E0_exact) / E0_exact)

E_iTEBD = -1.0635289694606316
E_exact = -1.063544409973372
de =  -1.4517976490299985e-05
